# utils & framework

In [2]:
from collections.abc import Iterable
from os import O_TEXT

import scipy.constants
import sympy
from IPython.core.display import Markdown
from sympy import *
from sympy.vector import *

ASSUM = sympy.Q

tag_id = 0
conditions = {}
values = {}
formulas_names = {}
restore_stack = []


def reset():
    global tag_id, conditions, values, formulas_names
    if restore_stack:
        state = restore_stack[-1]
        tag_id = state['tag_id']
        conditions = state['conditions'].copy()
        values = state['values'].copy()
        formulas_names = state['formulas_names'].copy()


def push_state():
    global restore_stack
    restore_stack.append({
        'tag_id': tag_id,
        'conditions': conditions.copy(),
        'values': values.copy(),
        'formulas_names': formulas_names.copy()
    })


def pop_state():
    global restore_stack
    return restore_stack.pop()


def save_as_condition(condition):
    global tag_id, conditions
    cached_condition_id = tag_id
    condition = spf(condition)
    conditions[tag_id] = condition
    tag_id += 1
    return cached_condition_id


def substitute_values(expression, mask=None):
    while True:
        left = False
        items = values.items() if mask is None else filter(lambda item: item[0] in mask, values.items())
        for (key, value) in items:
            before = str(expression)
            expression = expression.subs(key, value)
            expression = simplify(expression)
            if str(expression) != before:
                left = True
        if not left:
            break

    return expression


def save_as_variable(symbol, value):
    global values
    if symbol in values.keys():
        raise ValueError(f"variable {symbol} already exists")
    values[symbol] = substitute_values(S(value))


def save_as_formula_name(symbol, formula):
    global tag_id, formulas_names
    tag_id += 1
    if symbol in formulas_names.keys():
        raise ValueError(f"formula {symbol} already exists")
    formulas_names[symbol] = formula


def beautiful_print(expression, message="$", tag=None):
    expression = expression if isinstance(expression, str) else latex(expression)
    inline = not message.endswith('$')
    tag_str = f'\\tag{{{tag}}}' if tag is not None else ""
    if inline:
        full_latex = message + '$' + expression + '$'
    else:
        full_latex = message + '$' + expression + tag_str + '$$'
    display(Markdown(full_latex))


def beautiful_answer(expression, message="$", f: bool = False):
    symbol = expression
    expression = spf(expression)
    expression = expression.evalf() if f else expression
    beautiful_print(Eq(symbol, expression, evaluate=false), message, tag="answer")


def beautiful_inlined(expression, tag=None):
    expression = spf(expression)
    beautiful_print(expression, "inlined:$", tag)


def beautiful_peek(expression, message="$", tag=None):
    symbol= expression(False) if callable(expression) else expression
    value = spf(expression(True) if callable(expression) else expression)
    beautiful_print(Eq(symbol, value, evaluate=False), message, tag)


def beautiful_solve(cond, variables, apply: dict = None):
    cond = [cond] if not isinstance(cond, Iterable) else cond
    display(Markdown(f"${','.join(f"\\left ( {i} \\right ) " for i in cond)} \\Longrightarrow$"))

    variables = [variables] if not isinstance(variables, Iterable) else variables

    apply = apply if apply is not None else {}

    cond = tuple(map(lambda i: conditions[i].subs(apply), cond))
    cond = map(lambda i: substitute_values(i), cond)
    solution = solve(cond, tuple(variables), dict=True)[0]
    conclusions = []
    for (variable, conclusion) in solution.items():
        beautiful_print(Eq(variable, conclusion), "$")
        save_as_variable(variable, conclusion)
        conclusions.append(conclusion)


def beautiful_condition(expression, message="$"):
    beautiful_print(expression(False) if callable(expression) else expression, message, tag_id)
    return save_as_condition(expression(True) if callable(expression) else expression)


def beautiful_compute(symbol, expression, message="$", tag=None):
    symbol = symbols(symbol) if isinstance(symbol, str) else symbol
    global tag_id
    beautiful_print(Eq(symbol, expression(False) if callable(expression) else expression, evaluate=False), message,
                    tag_id if tag else None)
    if tag:
        tag_id += 1
    save_as_variable(symbol, expression(True) if callable(expression) else expression)
    return symbol


def beautiful_compute_at(symbol, expression, subs: dict, message="$", tag=None):
    symbol = symbols(symbol) if isinstance(symbol, str) else symbol
    global tag_id
    beautiful_print(Eq(symbol, expression.subs(subs), evaluate=False), message,
                    tag_id if tag else None)
    if tag:
        tag_id += 1
    save_as_variable(symbol, substitute_values(expression).subs(subs))
    return symbol


def beautiful_formula(symbol: str, expression: object, message: object = "$", ):
    beautiful_print(f"{symbol}=" + latex(expression), message, tag_id)
    save_as_formula_name(expression, symbol)
    return expression


def beautiful_at(formula, subs: dict, message="$", tag: bool = None):
    result = simplify(formula.subs(subs))
    param = '\\,'.join(f"{latex(item[0])}={latex(item[1])}" for item in subs.items())
    beautiful_print(formulas_names[formula] + f"\\bigg|_{{{param}}}=" + latex(result), message, tag)
    return result


def spf(expression):
    expression = substitute_values(expression)
    expression = simplify(expression)
    expression = expression.doit()
    return expression


class D:
    def __init__(self, symbol: Symbol, name: str = None):
        symbol = 'd' + latex(symbol) if name is None else name
        self.symbol = Symbol(symbol)

    def __enter__(self):
        return self.symbol

    def __exit__(self, type, value, traceback):
        save_as_variable(self.symbol, 1)


def map_vector(vec, func):
    return sum(
        # 0 is i/j/k sharp, 1 is expr
        map(lambda item: func(item[1]) * item[0], vec.components.items()),
        Vector.zero
    )


def series_vector(vec, param, x0=0, n=2):
    return map_vector(vec, lambda e: series(e, param, x0, n).removeO())


def integrate_vector(vec, param):
    return map_vector(vec, lambda e: integrate(e, param))


def md(msg):
    display(Markdown(msg))


def spt():
    md("-----------------")


def beautiful_vector(xi=0, yj=0, zk=0):
    xi = Symbol(xi) if isinstance(xi, str) else xi
    yj = Symbol(yj) if isinstance(yj, str) else yj
    zk = Symbol(zk) if isinstance(zk, str) else zk
    return xi * coord.i + yj * coord.j + zk * coord.k


def beautiful_branch_at(expression, index):
    expression = spf(expression)
    branch, cond = expression.args[index]
    return branch


def refine_piecewise(pw_expr, assume):
    pw_expr = spf(pw_expr)

    for expr_val, cond in pw_expr.args:
        if ask(cond, assume):
            return refine(expr_val, assume)

    # If no branch can be definitively chosen, refine each branch
    refined_args = [(refine(expr_val, assume), cond)
                    for expr_val, cond in pw_expr.args]
    return Piecewise(*refined_args)


coord = CoordSys3D('')

## Common

In [3]:
epsilon_0 = symbols('epsilon_0', positive=True)
k = 1 / (4 * pi * epsilon_0)
charge = symbols('q')
x = coord.x
y = coord.y
z = coord.z
pos = x * coord.i + y * coord.j + z * coord.k
E = beautiful_formula('E', k * charge * pos / pos.magnitude() ** 3, "by Coulomb's law:$")
electric_flux = Symbol('phi_e')
enclosed_charge_sum = Symbol('Q_{sum}')
Gauss_law = beautiful_condition(Eq(electric_flux, enclosed_charge_sum / epsilon_0), "by Gauss's law:$")


def fn_potential(q, r):
    return k * q / r


P = beautiful_formula('\\varphi', fn_potential(charge, pos.magnitude()), 'electric potential:$')


def save_common_consts():
    save_as_variable(epsilon_0, scipy.constants.epsilon_0)
    save_as_variable(pi, scipy.constants.pi)


push_state()

by Coulomb's law:$$E=\left(\frac{\mathbf{{x}_{}} q}{4 \pi \epsilon_{0} \left(\mathbf{{x}_{}}^{2} + \mathbf{{y}_{}}^{2} + \mathbf{{z}_{}}^{2}\right)^{\frac{3}{2}}}\right)\mathbf{\hat{i}_{}} + \left(\frac{\mathbf{{y}_{}} q}{4 \pi \epsilon_{0} \left(\mathbf{{x}_{}}^{2} + \mathbf{{y}_{}}^{2} + \mathbf{{z}_{}}^{2}\right)^{\frac{3}{2}}}\right)\mathbf{\hat{j}_{}} + \left(\frac{\mathbf{{z}_{}} q}{4 \pi \epsilon_{0} \left(\mathbf{{x}_{}}^{2} + \mathbf{{y}_{}}^{2} + \mathbf{{z}_{}}^{2}\right)^{\frac{3}{2}}}\right)\mathbf{\hat{k}_{}}\tag{0}$$

by Gauss's law:$$\phi_{e} = \frac{Q_{sum}}{\epsilon_{0}}\tag{1}$$

electric potential:$$\varphi=\frac{q}{4 \pi \epsilon_{0} \sqrt{\mathbf{{x}_{}}^{2} + \mathbf{{y}_{}}^{2} + \mathbf{{z}_{}}^{2}}}\tag{2}$$

In [6]:
reset()
def t72():
    m_1,m_2,m_3 = symbols('m_1 m_2 m_3')
    r , omega = symbols('r omega')
    t = symbols('t')
    y= beautiful_compute(Function('y')(t),r*sin(omega*t))
    F= Function('F')(t)
    g = symbols('g')
    c1 = beautiful_condition(Eq(-F-(m_1+m_2+m_3)*g, diff(y,t,2)*m_3+ diff(y,t,2)* m_2/2))
    beautiful_solve(c1,F)
t72()

$$y{\left(t \right)} = r \sin{\left(\omega t \right)}$$

$$- g \left(m_{1} + m_{2} + m_{3}\right) - F{\left(t \right)} = \frac{m_{2} \frac{d^{2}}{d t^{2}} y{\left(t \right)}}{2} + m_{3} \frac{d^{2}}{d t^{2}} y{\left(t \right)}\tag{3}$$

$\left ( 3 \right )  \Longrightarrow$

$$F{\left(t \right)} = - g m_{1} - g m_{2} - g m_{3} + \frac{m_{2} \omega^{2} r \sin{\left(\omega t \right)}}{2} + m_{3} \omega^{2} r \sin{\left(\omega t \right)}$$

In [11]:

reset()
def t73():

    m_1,m_2,m_3 = symbols('m_1 m_2 m_3')
    r , omega = symbols('e omega')
    t = symbols('t')
    y= beautiful_compute(Function('x')(t),r*cos(omega*t))
    F= Function('F_x')(t)
    g = symbols('g')
    c1 = beautiful_condition(Eq(F, diff(y,t,2)*m_1+ diff(y,t,2)* m_2))
    beautiful_solve(c1,F)
    F= Function('F_y')(t)
    y= beautiful_compute(Function('y')(t),r*sin(omega*t))
    c1 = beautiful_condition(Eq(F, diff(y,t,2)*m_1))
    beautiful_solve(c1,F)

t73()

$$x{\left(t \right)} = e \cos{\left(\omega t \right)}$$

$$F_{x}{\left(t \right)} = m_{1} \frac{d^{2}}{d t^{2}} x{\left(t \right)} + m_{2} \frac{d^{2}}{d t^{2}} x{\left(t \right)}\tag{3}$$

$\left ( 3 \right )  \Longrightarrow$

$$F_{x}{\left(t \right)} = - e m_{1} \omega^{2} \cos{\left(\omega t \right)} - e m_{2} \omega^{2} \cos{\left(\omega t \right)}$$

$$y{\left(t \right)} = e \sin{\left(\omega t \right)}$$

$$F_{y}{\left(t \right)} = m_{1} \frac{d^{2}}{d t^{2}} y{\left(t \right)}\tag{4}$$

$\left ( 4 \right )  \Longrightarrow$

$$F_{y}{\left(t \right)} = - e m_{1} \omega^{2} \sin{\left(\omega t \right)}$$

In [14]:
reset()
def t74():
   x= symbols('x')
   xb= symbols('x_b')
   R , omega = symbols('R omega')

   m_1,m_2,m_3 = symbols('m_1 m_2 m_3')
   y= symbols('y')
   c1=beautiful_condition(Eq(x*m_2+xb*m_1,0))
   c2=beautiful_condition(Eq((x-xb)**2+ y**2,R**2))
   beautiful_solve([c1,c2],[y,xb])

t74()

$$m_{1} x_{b} + m_{2} x = 0\tag{3}$$

$$y^{2} + \left(x - x_{b}\right)^{2} = R^{2}\tag{4}$$

$\left ( 3 \right ) ,\left ( 4 \right )  \Longrightarrow$

$$x_{b} = - \frac{m_{2} x}{m_{1}}$$

$$y = - \frac{\sqrt{- \left(- R m_{1} + m_{1} x + m_{2} x\right) \left(R m_{1} + m_{1} x + m_{2} x\right)}}{m_{1}}$$

In [34]:
reset()
def t75():

    m,M = symbols('m M')
    alpha,k = symbols('alpha k')
    t = symbols('t')
    x_1,x_2 = Function('x_1')(t), Function('x_2')(t)
    c1= beautiful_condition(Eq(x_1*m + x_2*M,0))
    g= symbols('g')
    beautiful_solve(c1,x_2)
    c2 = beautiful_condition(Eq((x_2-x_1)*tan(alpha)*m*g-((x_2-x_1)/cos(alpha))**2/2*k,m*diff(x_1,t)**2/2+ M*diff(x_2,t)**2/2))
    beautiful_solve(c2,diff(x_1,t)**2)

t75()

$$M x_{2}{\left(t \right)} + m x_{1}{\left(t \right)} = 0\tag{3}$$

$\left ( 3 \right )  \Longrightarrow$

$$x_{2}{\left(t \right)} = - \frac{m x_{1}{\left(t \right)}}{M}$$

$$g m \left(- x_{1}{\left(t \right)} + x_{2}{\left(t \right)}\right) \tan{\left(\alpha \right)} - \frac{k \left(- x_{1}{\left(t \right)} + x_{2}{\left(t \right)}\right)^{2}}{2 \cos^{2}{\left(\alpha \right)}} = \frac{M \left(\frac{d}{d t} x_{2}{\left(t \right)}\right)^{2}}{2} + \frac{m \left(\frac{d}{d t} x_{1}{\left(t \right)}\right)^{2}}{2}\tag{4}$$

$\left ( 4 \right )  \Longrightarrow$

$$\left(\frac{d}{d t} x_{1}{\left(t \right)}\right)^{2} = - \frac{g x_{1}{\left(t \right)} \sin{\left(2 \alpha \right)}}{\cos^{2}{\left(\alpha \right)}} - \frac{k x_{1}^{2}{\left(t \right)}}{m \cos^{2}{\left(\alpha \right)}} - \frac{k x_{1}^{2}{\left(t \right)}}{M \cos^{2}{\left(\alpha \right)}}$$